## OBP:

Afspraken:

-   [ ] We gaan verder kijken naar XG Boosting
-   [ ] Data: optimale schedules, labels: waiting times en tardiness

Idee Joost: cardinal analysis - is schedule a better than b, yes/no


In [ ]:
from schedule_class import Schedule, generate_schedules
import numpy as np
import pandas as pd
import plotly.express as px
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Example usage:
x = np.array([2, 1, 0])
d = 3
s = [0.1, 0.3, 0.25, 0.2, 0.15]
q = 0.0
omega = 0.5

schedule = Schedule(x, d, s, q, omega)
print(schedule)

In [ ]:
def run_schedule(x, d, s, q, omega, print_system=True):
    schedule = Schedule(x=x, d=d, s=s, q=q, omega=omega)
    schedule.calculate_system_states(until=len(x))
    schedule.calculate_wait_times()
    if(print_system): print(schedule)
    return(schedule)

In [ ]:
"""Simple test case"""
d = 3
q = 0.0
omega = 0.6
s=np.array(
    [0.1, 0.2, 0.3, 0.2, 0.15, 0.05, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float64)
# s= service_time_with_no_shows(s, q)
indices = np.arange(s.size)
exp_s = (indices * s).sum()
print(f'exp_s = {exp_s}')

In [ ]:
N = 6
schedules = generate_schedules(N) # Generates all possible schedules with T hard-coded 3

data = {'x0': [], 'x1': [], 'x2': [], 'loss': []}
for schedule in schedules:
  x = np.array(schedule, dtype=np.int64)
  sch = run_schedule(x, d, s, q, omega, False)
  x0, x1, x2 = x
  data['x0'].append(x0)
  data['x1'].append(x1)
  data['x2'].append(x2)
  data['loss'].append(sch.system['loss'])

df = pd.DataFrame.from_dict(data).sort_values(by=['loss'], ascending=False)
df

In [ ]:
N = 18
data = {'x0': [], 'x1': [], 'x2': [], 'ew0': [], 'ew1': [], 'ew2': []}
df = pd.DataFrame.from_dict(data)

for n in range(1,N+1):
    schedules = generate_schedules(n) # Generates all possible schedules with T hard-coded 3
    for schedule in schedules:
      x = np.array(schedule, dtype=np.int64)
      sch = run_schedule(x, d, s, q, omega, False)
      x0, x1, x2 = x
      data['x0'].append(x0)
      data['x1'].append(x1)
      data['x2'].append(x2)
      data['ew0'].append(sch.system['ew'][0])
      data['ew1'].append(sch.system['ew'][1])
      data['ew2'].append(sch.system['ew'][2])
      
      # Convert the current data dictionary to a DataFrame and append it to the main DataFrame
      temp_df = pd.DataFrame.from_dict(data)
      df = pd.concat([df, temp_df])
      data = {'x0': [], 'x1': [], 'x2': [], 'ew0': [], 'ew1': [], 'ew2': []}

df

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split data into features and labels
X = df[['x0', 'x1', 'x2']]
y1 = df['ew0']
y2 = df['ew1']
y3 = df['ew2']

# Split into training and testing datasets
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)
_, _, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)
_, _, y3_train, y3_test = train_test_split(X, y3, test_size=0.2, random_state=42)

# Train models
params = {
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'eta': 0.1,
    'silent': 1
}
num_round = 100

# Model for output_1
dtrain1 = xgb.DMatrix(X_train.iloc[:, 0], label=y1_train)
dtest1 = xgb.DMatrix(X_test.iloc[:, 0], label=y1_test)
model1 = xgb.train(params, dtrain1, num_round)

# Model for output_2
dtrain2 = xgb.DMatrix(X_train.iloc[:, 0:2], label=y2_train)
dtest2 = xgb.DMatrix(X_test.iloc[:, 0:2], label=y2_test)
model2 = xgb.train(params, dtrain2, num_round)

# Model for output_3
dtrain3 = xgb.DMatrix(X_train, label=y3_train)
dtest3 = xgb.DMatrix(X_test, label=y3_test)
model3 = xgb.train(params, dtrain3, num_round)

In [ ]:
# Make predictions
preds1 = model1.predict(dtest1)
preds2 = model2.predict(dtest2)
preds3 = model3.predict(dtest3)

# Combine predictions into a single output vector for each input
predictions = pd.DataFrame({
    'output_1': preds1,
    'output_2': preds2,
    'output_3': preds3
})

print(predictions)

In [ ]:
mse1 = mean_squared_error(y1_test, preds1)
mse2 = mean_squared_error(y2_test, preds2)
mse3 = mean_squared_error(y3_test, preds3)

print(f'MSE for output_1: {mse1}')
print(f'MSE for output_2: {mse2}')
print(f'MSE for output_3: {mse3}')

In [ ]:
# Create a DataFrame with actual and predicted values for comparison
comparison_df = pd.DataFrame({
    'Actual_output_1': y1_test.values,
    'Predicted_output_1': preds1,
    'Actual_output_2': y2_test.values,
    'Predicted_output_2': preds2,
    'Actual_output_3': y3_test.values,
    'Predicted_output_3': preds3
})

# Create scatter plots for each output
fig1 = px.scatter(comparison_df, x='Actual_output_1', y='Predicted_output_1', title='Actual vs Predicted - Output 1')
fig2 = px.scatter(comparison_df, x='Actual_output_2', y='Predicted_output_2', title='Actual vs Predicted - Output 2')
fig3 = px.scatter(comparison_df, x='Actual_output_3', y='Predicted_output_3', title='Actual vs Predicted - Output 3')

# Show the plots
fig1.show()
fig2.show()
fig3.show()